In [11]:
import numpy as np
import os
import tensorflow as tf

tf.reset_default_graph() # 그래프 초기화(변수생성)
n_inputs = 3 # 입력 데이터
n_neurons = 5 # 셀의 가중치 사이즈

X0 = tf.placeholder(tf.float32, [None, n_inputs]) # 4x3
X1 = tf.placeholder(tf.float32, [None, n_inputs]) # 4x3
# [0, 1, 2] 사이즈가 하나의 셀로 입력
# FFNN(feed forward neural network)
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons) # 셀을 만들면서 가중치 사이즈(특성을 찾아내는 것)
# static_rnn : rnn network(4개의 셀이 연결되면서 메모리 확보)
output_seqs, states = tf.contrib.rnn.static_rnn(basic_cell, [X0, X1], dtype = tf.float32) # 2(배치사이즈) x 4 (셀수)x 3(뉴런수)
# states : 다음으로 연결되어지는 값
Y0, Y1 = output_seqs
# 출력, 다음 셀로 전달되어지는 값(마지막 states값 -> 수평으로 셀을 연결)
init = tf.global_variables_initializer()
X0_batch = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 0, 1]]) # 셀 하나씩 먼저 계산하고 옆으로 넘기고 그걸 가지고 또 계산하고 넘기는 구조
X1_batch = np.array([[9, 8, 7], [0, 0, 0], [6, 5, 4], [3, 2, 1]])

In [19]:
with  tf.Session() as sess :
    init.run()
    Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict={X0:X0_batch, X1 : X1_batch})
print("처음 데이터 특성 : \n", Y0_val, "\n차수 : ", Y0_val.shape)
print("===============================================================================")
print("두번째 데이터 특성 :\n ", Y1_val, "\n차수 : ", Y1_val.shape)
# 4x3 => 4x5

처음 데이터 특성 : 
 [[ 0.8461064   0.51896864 -0.890019   -0.84969074 -0.7028126 ]
 [ 0.9999839   0.04929762 -0.99457866 -0.9996834  -0.99962807]
 [ 1.         -0.44323686 -0.9997461  -0.9999996  -0.9999996 ]
 [ 0.999983   -0.9910994   0.9990058  -0.8651231  -0.9999922 ]] 
차수 :  (4, 5)
두번째 데이터 특성 :
  [[ 1.         -0.95148987 -0.98448163 -0.9999976  -1.        ]
 [ 0.02512543  0.20363319  0.60816026  0.8447146  -0.9013321 ]
 [ 0.9999999  -0.9181077  -0.7767207  -0.9992909  -0.99999917]
 [ 0.99946105 -0.06703981 -0.36780044 -0.96766204 -0.997534  ]] 
차수 :  (4, 5)


# 셀이 가지고 있는 가중치 사이즈는 3*5 (3에서 5로 변하기 때문)

In [25]:
# 이미지 784(28x28)
n_steps = 28    # 셀수
n_inputs = 28   # 셀당 인풋사이즈
n_neurons = 150 # 뉴런 출력
n_outputs = 10  # 확률 사이즈 
learning_rate = 0.001
tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs]) # 3차원으로 [배치사이즈, 셀수, 셀당 입력수]
y = tf.placeholder(tf.int32, [None])
# 셀 하나가 FFNN
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units = n_neurons) # 150
# 자동으로 셀 : 28
# state 마지막 셀의 수평으로 전달되는 값
# state는 마지막 셀의 output과 같다.
# 28개의 셀이 있는데 마지막 1개의 output을 사용 -> many to one
# 감정분류 => 분류기
# state의 차수 : 150x150
# output의 차수 : latent time 지연시간을 통해 계산된 셀의 모든 값을 결합해서 출력 150*28*150
output, states = tf.nn.dynamic_rnn(basic_cell, X, dtype = tf.float32)
# 150개의 특징중 10개만 추출
# dense : 입력차수와 출력차수만 지정하면 자동으로 바이어스 생성, 가중치  공간을 확보해서
# 150x150 => 150x10
# dense의 가중치 사이즈 = 150*10
logits = tf.layers.dense(states, n_outputs) 

xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = logits) # 분류를 위한 미분이 가능한 식
loss = tf.reduce_mean(xentropy) # 배치 사이즈평균을 통해서 구함
# Adam = momentun + propgrad 
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate) # 경사하강법
training_op = optimizer.minimize(loss)
correct = tf.nn.in_top_k(logits, y, 1) # 가장 큰 값을 취해서 비교
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
init = tf.global_variables_initializer()

In [26]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")
X_test = mnist.test.images.reshape((-1, n_steps, n_inputs)) # 원래 이미지 모양으로 학습
y_test = mnist.test.labels

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [45]:
print(mnist.test.num_examples)
print(mnist.train.num_examples)

10000
55000


In [27]:
n_epochs = 100
batch_size = 150
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs) :
        for iteration in range(mnist.train.num_examples // batch_size) : # 60000/150
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            X_batch = X_batch.reshape((-1, n_steps, n_inputs))
            sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
        acc_train = accuracy.eval(feed_dict={X:X_batch, y:y_batch})
        acc_test = accuracy.eval(feed_dict={X:X_test, y:y_test})
        
        print(epoch, "\tTrain acuuracy : ", acc_train, "\tTest accuracy : ", acc_test)
        

0 	Train acuuracy :  0.8933333 	Test accuracy :  0.9215
1 	Train acuuracy :  0.94666666 	Test accuracy :  0.9337
2 	Train acuuracy :  0.97333336 	Test accuracy :  0.9505
3 	Train acuuracy :  0.96 	Test accuracy :  0.9583
4 	Train acuuracy :  0.98 	Test accuracy :  0.953
5 	Train acuuracy :  0.94666666 	Test accuracy :  0.9642
6 	Train acuuracy :  0.98 	Test accuracy :  0.9667
7 	Train acuuracy :  0.98 	Test accuracy :  0.9684
8 	Train acuuracy :  0.9866667 	Test accuracy :  0.9683
9 	Train acuuracy :  0.97333336 	Test accuracy :  0.9688
10 	Train acuuracy :  0.99333334 	Test accuracy :  0.9756
11 	Train acuuracy :  0.98 	Test accuracy :  0.9743
12 	Train acuuracy :  0.9866667 	Test accuracy :  0.9689
13 	Train acuuracy :  0.99333334 	Test accuracy :  0.9736
14 	Train acuuracy :  1.0 	Test accuracy :  0.97
15 	Train acuuracy :  0.97333336 	Test accuracy :  0.9611
16 	Train acuuracy :  0.97333336 	Test accuracy :  0.9731
17 	Train acuuracy :  0.97333336 	Test accuracy :  0.9733
18 	Train

# multi rnn cell

In [51]:
# BasicRNNCell : 기본데이터가 입력
# MultiRNNCell : 수직으로 레이어 구성 -> 수직으로 셀이 구성
# dynamic_rnn : 모델
tf.reset_default_graph()
n_steps = 28
n_inputs = 28
n_outputs = 10
learning_rate = 0.001
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.int32, [None])
n_layers = 3 # 3개의 멀티 레이어 -> 3개의 셀을 생성
layers = [tf.contrib.rnn.BasicRNNCell(num_units = n_neurons, activation=tf.nn.relu)
         for layer in range(n_layers)]
# 3개의 셀을 조합해서 MultiRNNCell
multi_layer_cell = tf.contrib.rnn.MultiRNNCell(layers)

In [52]:
# dynamic_rnn : 셀로 입력되는 데이터의 개수에 맞추어서 셀을 구성
# 나는 학교에 간다. (2자 3자 2자) -> 사이즈가 다르기 때문에 
# state_rnn에서는 동일한 사이즈를 만드려고 함 : 큰것 기준, 작은 것은 padding, 작으면 안좋아지기 때문에 dynamic_rnn사용
# 입력사이즈 변동 => 가중치 조절 -> 나가는 특징은 일치
# 3층 셀이 28개가 조성 : 고정 사이즈
outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype = tf.float32)
# states가 몇개 발생하는가
# 3x150x100(neuron)
states_concat = tf.concat(axis = 1, values = states) # 150 x 300
# 가중치 학습 FFNN 망
logits = tf.layers.dense(states_concat, n_outputs)   # 가중치 : 300x10
# 150(미니배치)x10(확률값)
# 원핫인코딩 * log(예측값)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits = logits) 
loss = tf.reduce_mean(xentropy)
optimizer =  tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
init = tf.global_variables_initializer()

In [53]:
n_epochs = 100
batch_size = 150
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs) :
        for iteration in range(mnist.train.num_examples // batch_size) : # 60000/150
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            X_batch = X_batch.reshape((-1, n_steps, n_inputs))
            sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
        acc_train = accuracy.eval(feed_dict={X:X_batch, y:y_batch})
        acc_test = accuracy.eval(feed_dict={X:X_test, y:y_test})
        
        print(epoch, "\tTrain acuuracy : ", acc_train, "\tTest accuracy : ", acc_test)
        

0 	Train acuuracy :  0.96666664 	Test accuracy :  0.9544
1 	Train acuuracy :  0.97333336 	Test accuracy :  0.9737
2 	Train acuuracy :  0.97333336 	Test accuracy :  0.9714
3 	Train acuuracy :  0.99333334 	Test accuracy :  0.9754
4 	Train acuuracy :  0.9866667 	Test accuracy :  0.9756
5 	Train acuuracy :  0.98 	Test accuracy :  0.9779
6 	Train acuuracy :  1.0 	Test accuracy :  0.9832
7 	Train acuuracy :  0.97333336 	Test accuracy :  0.9799
8 	Train acuuracy :  1.0 	Test accuracy :  0.9833
9 	Train acuuracy :  0.99333334 	Test accuracy :  0.9851
10 	Train acuuracy :  0.99333334 	Test accuracy :  0.9844
11 	Train acuuracy :  1.0 	Test accuracy :  0.9836
12 	Train acuuracy :  0.9866667 	Test accuracy :  0.9855
13 	Train acuuracy :  0.99333334 	Test accuracy :  0.9834
14 	Train acuuracy :  0.99333334 	Test accuracy :  0.9805
15 	Train acuuracy :  0.99333334 	Test accuracy :  0.9814
16 	Train acuuracy :  0.99333334 	Test accuracy :  0.9828
17 	Train acuuracy :  1.0 	Test accuracy :  0.9803
18

In [55]:
import tensorflow as tf
from tensorflow.python.framework import ops
ops.reset_default_graph()
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot = True)
learning_rate = 0.001
total_epoch = 10
batch_size = 128
n_input = 28
n_step = 28
n_hidden = 128
n_class = 10
X = tf.placeholder(tf.float32, [None, n_step, n_input]) # 128x28x28
y = tf.placeholder(tf.float32, [None, n_class]) # 128x10
W = tf.Variable(tf.random_normal([n_hidden, n_class])) # 128x10
b = tf.Variable(tf.random_normal([n_class])) # 10

cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden) # 28x128 사이즈의 가중치 # neurons : 128특징
# cell 수 : 28개
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype = tf.float32)
# 128x28x128,   128x128
# FFNN  
outputs = tf.transpose(outputs, [1,0,2]) # 28x128x128
# 셀로 판단
outputs = outputs[-1] # 맨 마지막 셀 : 128x128
model = tf.matmul(outputs, W) + b# 128x128 128x10 => 128x10
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels = y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [62]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
total_batch = int(mnist.train.num_examples/batch_size)
for epoch in range(total_epoch) :
    total_cost = 0
    for i in range(total_batch) :
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        # 이미지 사이즈로 리셋
        batch_xs = batch_xs.reshape((batch_size, n_step, n_input))
        # python이 계산한 마지막 결과가 저장되는 것이 _
        _, cost_val = sess.run([optimizer, cost], feed_dict={X:batch_xs, y:batch_ys})
        total_cost += cost_val
    print('Epoch', '%04d' %(epoch+1),
         'Avg.cost = ', '{:.3f}'.format(total_cost/total_batch))
print("최적화 완료")

Epoch 0001 Avg.cost =  0.544
Epoch 0002 Avg.cost =  0.237
Epoch 0003 Avg.cost =  0.176
Epoch 0004 Avg.cost =  0.152
Epoch 0005 Avg.cost =  0.135
Epoch 0006 Avg.cost =  0.128
Epoch 0007 Avg.cost =  0.118
Epoch 0008 Avg.cost =  0.111
Epoch 0009 Avg.cost =  0.101
Epoch 0010 Avg.cost =  0.101
최적화 완료


# 문제
1. 테스트 데이터를 이용하여 테스트하는 회로를 구성하시요
2. 3개의 셀을 갖늕 multi-layer cell로 수정하시요

In [64]:
tf.reset_default_graph()
n_steps = 28
n_inputs = 28
n_outputs = 10
learning_rate = 0.001
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.int32, [None])
n_layers = 3
layers = [tf.contrib.rnn.BasicRNNCell(num_units = n_neurons, activation=tf.nn.relu)
         for layer in range(n_layers)]

multi_layer_cell = tf.contrib.rnn.MultiRNNCell(layers)

outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype = tf.float32)

states_concat = tf.concat(axis = 1, values = states) 
logits = tf.layers.dense(states_concat, n_outputs)


xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits = logits) 
loss = tf.reduce_mean(xentropy)
optimizer =  tf.test.AdamOptimizer(learning_rate=learning_rate)
test_op = optimizer.minimize(loss)
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
init = tf.global_variables_initializer()

n_epochs = 100
batch_size = 150
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs) :
        for iteration in range(mnist.test.num_examples // batch_size) : # 60000/150
            X_batch, y_batch = mnist.test.next_batch(batch_size)
            X_batch = X_batch.reshape((-1, n_steps, n_inputs))
            sess.run(test_op, feed_dict={X:X_batch, y:y_batch})
        acc_train = accuracy.eval(feed_dict={X:X_batch, y:y_batch})
        acc_test = accuracy.eval(feed_dict={X:X_test, y:y_test})
        
        print(epoch, "\tTrain acuuracy : ", acc_train, "\tTest accuracy : ", acc_test)
        

AttributeError: module 'tensorflow._api.v1.test' has no attribute 'AdamOptimizer'

In [67]:
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
test_batch_size = len(mnist.test.images)
test_xs =  mnist.test.images.reshape(test_batch_size, n_step, n_input)
test_ys = mnist.test.labels
print('정확도 : ', sess.run(accuracy, feed_dict={X: test_xs, y:test_ys}))

ValueError: Tensor("ArgMax:0", shape=(), dtype=int64) must be from the same graph as Tensor("ArgMax_1:0", shape=(?,), dtype=int64).

In [69]:
import tensorflow as tf
from tensorflow.python.framework import ops
ops.reset_default_graph()
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot = True)
learning_rate = 0.001
total_epoch = 10
batch_size = 128
n_input = 28
n_step = 28
n_hidden = 128
n_class = 10

# None이 batch_size
X = tf.placeholder(tf.float32, [None, n_step, n_input]) # 128x28x28
Y = tf.placeholder(tf.float32, [None, n_class]) # 128x10
W = tf.Variable(tf.random_normal([n_hidden, n_class])) # 128*10
b = tf.Variable(tf.random_normal([n_class]))  # 10
# 28x128 사이즈의 가중치

n_layers = 3
# 3개의 셀로 멀티레이어를 생성해도 output은 하나의 셀일 경우와 같다.
# state에서는 3개의 레이어마다 생성되기 때문에
layers = [tf.contrib.rnn.BasicRNNCell(num_units=n_hidden, activation =tf.nn.relu)
         for layer in range(n_layers)]
multi_layer_cell = tf.contrib.rnn.MultiRNNCell(layers)

cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden) # RNNCell => neurons : 128개의 특징
# cell 수 : 28
outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype = tf.float32)
# 128x28x128, 128x128
# FFNN
outputs = tf.transpose(outputs, [1,0,2]) # 28x128x128
# 셀로
outputs = outputs[-1] # 맨마지막 셀 : 128x128
model = tf.matmul(outputs, W) + b # 128x128 , 가중치 128x10 => 128x10
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels = Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [74]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
is_correct = tf.equal(tf.argmax(model,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
test_batch_size = len(mnist.test.images)
test_xs = mnist.test.images.reshape(test_batch_size, n_step,n_input)
test_ys = mnist.test.labels
# print("정확도", sess.run(accuracy), feed_dict={X:test_xs, Y:test_ys})
print("정확도:", sess.run(accuracy, feed_dict={X: test_xs, Y: test_ys}))

정확도: 0.1102


In [75]:
import numpy as np
values231 = np.array([
    [[1],[2],[3]],
    [[2],[3],[4]]
])
print("입력데이터의 shape", values231.shape) # 2 x 3x 1
import tensorflow as tf
tf.reset_default_graph()
tf_values231 = tf.constant(values231, dtype = tf.float32)
# 1x100
lstm_cell = tf.contrib.rnn.LSTMCell(num_units=100) # 뉴런
# rnn모델은 동일
outputs, state = tf.nn.dynamic_rnn(cell=lstm_cell, dtype=tf.float32, inputs=tf_values231)
print("rnn이 출력하는 output의 의미 : ", outputs.shape) # 2x3x100
with tf.Session() as sess :
    sess.run(tf.global_variables_initializer())
    output_run, state_run = sess.run([outputs, state])

입력데이터의 shape (2, 3, 1)
rnn이 출력하는 output의 의미 :  (2, 3, 100)


In [76]:
tf.reset_default_graph()
values = tf.constant(np.array([
    [[1],[2],[3]],
    [[2],[3],[4]]
]), dtype = tf.float32)
lstm_cell_fw = tf.contrib.rnn.LSTMCell(100)
lstm_cell_bw = tf.contrib.rnn.LSTMCell(100)
# Multi Cell : 셀에서
# 양방향 LSTM
(output_fw, output_bw), (output_state_fw, output_state_bw) = tf.nn.bidirectional_dynamic_rnn(
    cell_fw=lstm_cell_fw,
    cell_bw=lstm_cell_bw,
    inputs=values,
    dtype=tf.float32)
print(output_fw.shape)
print(output_bw.shape)

Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
(2, 3, 100)
(2, 3, 100)


In [79]:
# embedding
from tensorflow.contrib import rnn
import pprint
from tensorflow.python.framework import ops
ops.reset_default_graph()
pp = pprint.PrettyPrinter(indent=4)
sess = tf.InteractiveSession()
h = [1, 0, 0, 0]   # 단어를 원핫인코딩 : 수치화
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]
# 변수공유를 위해 지정
with tf.variable_scope('five_sequences') as scope :
    hidden_size = 2
    # 4->2개 특징으로 vector화
    cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
    # [h,e,l,l,o]
    # [1, 0, 0, 0]
    # [0, 1, 0, 0]
    # [0, 0, 1, 0]
    # [0, 0, 1, 0]
    # [0, 0, 0, 1]
    
    x_data = np.array([[h,e,l,l,o]], dtype = np.float32)
    print(x_data.shape) # (1, 5, 4) : 데이터 한개 
    pp.pprint(x_data)
    # cell의 개수 : 5개
    # 셀당 입력데이터 4개
    # 셀당 가중치 : 4x2
    outputs, states = tf.nn.dynamic_rnn(cell, x_data, dtype = tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())
    print(outputs.shape) # 1x5x2
    print(states.shape)  # 1x2
    pp.pprint(states.eval())

(1, 5, 4)
array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]], dtype=float32)
array([[[ 0.69816405, -0.23959392],
        [ 0.56656075, -0.4680605 ],
        [ 0.34961826,  0.66894406],
        [ 0.16789563,  0.49183697],
        [ 0.61552846, -0.7606419 ]]], dtype=float32)
(1, 5, 2)
(1, 2)
array([[ 0.61552846, -0.7606419 ]], dtype=float32)


In [80]:
char_arr = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
num_dic = {n: i for i, n in enumerate(char_arr)}
# a:0, b:1
dic_len = len(num_dic)
# rule base => 자동 툴 => 가중치만 이용
seq_data = ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']
# 10x3
# 10x2 target

In [81]:
# 26x26행렬

# 100000000000000000000000
# 010000000000000000000000
# 001000000000000000000000

def make_batch(seq_data) :
    input_batch = []
    target_batch = []
    for seq in seq_data : # 'word'
        input = [num_dic[n] for n in seq[:-1]] # w=22, o = 14, r=17
        target = num_dic[seq[-1]] # d = 3
        input_batch.append(np.eye(dic_len)[input]) # 원핫인코딩
        target_batch.append(target)
    return input_batch, target_batch

In [82]:
learning_rate = 0.01
n_hidden = 128
total_epoch = 30
n_step = 3
n_input = n_class = dic_len
 
X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.float32, [None])
W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))